In [227]:
import cv2
import re

In [228]:
image_path = r"data/train/images/246-ktp_jpg.rf.fa9bf940931b0387b060f828f5ba6f8e.jpg"

# Convert image to RGB
img = cv2.imread(image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [229]:
# from doctr.models import ocr_predictor
# from doctr.io import DocumentFile

In [230]:
# model = ocr_predictor(det_arch='fast_small',reco_arch='crnn_mobilenet_v3_small',pretrained=True)

In [231]:
# single_img_doc = DocumentFile.from_images(image_path)
# result = model(single_img_doc)
# export = result.export()

In [232]:
# export

In [233]:
# export['pages'][0]["blocks"][0]["lines"]

In [234]:
import easyocr

reader = easyocr.Reader(['id'])
export = reader.readtext(image_path)

In [235]:
export

[([[np.int32(176), np.int32(73)],
   [np.int32(482), np.int32(73)],
   [np.int32(482), np.int32(106)],
   [np.int32(176), np.int32(106)]],
  'PROVINSI SULAWESI SELATAN',
  np.float64(0.9580166371000649)),
 ([[np.int32(223), np.int32(99)],
   [np.int32(433), np.int32(99)],
   [np.int32(433), np.int32(135)],
   [np.int32(223), np.int32(135)]],
  'KABUPATEN MAROS',
  np.float64(0.9961174373559115)),
 ([[np.int32(34), np.int32(140)],
   [np.int32(88), np.int32(140)],
   [np.int32(88), np.int32(172)],
   [np.int32(34), np.int32(172)]],
  'NIK',
  np.float64(0.9756025764265064)),
 ([[np.int32(164), np.int32(137)],
   [np.int32(433), np.int32(137)],
   [np.int32(433), np.int32(178)],
   [np.int32(164), np.int32(178)]],
  '7309012705930004',
  np.float64(0.9896844845657495)),
 ([[np.int32(33), np.int32(187)],
   [np.int32(86), np.int32(187)],
   [np.int32(86), np.int32(214)],
   [np.int32(33), np.int32(214)]],
  'Nama',
  np.float64(0.959918737411499)),
 ([[np.int32(178), np.int32(188)],
   [n

In [236]:
# def extract_objectness_scores(blocks):
#     scores = []
#     for block in blocks:
#         scores.append(block.get('objectness_score', 0))
#         if 'lines' in block:
#             for line in block['lines']:
#                 scores.append(line.get('objectness_score', 0))
#                 if 'words' in line:
#                     for word in line['words']:
#                         scores.append(word.get('objectness_score', 0))
#         if 'words' in block:
#             for word in block['words']:
#                 scores.append(word.get('objectness_score', 0))
#     return scores

# # Extract scores
# scores = extract_objectness_scores(export['pages'][0]['blocks'])

# # Calculate average
# average_score = sum(scores) / len(scores) if scores else 0
# average_score

In [237]:
class OCRTextProcessor:
    def __init__(self, tolerance=15):
        self.tolerance = tolerance
    
    def process_ocr_result(self, result):
        """Process OCR result and return corrected text alignments."""
        # Extract all words with their coordinates and text
        words = []
        # for page in result['pages']:
        #     for block in page['blocks']:
        #         for line in block.get('lines', []):
        #             for word in line.get('words', []):
        #                 # Get coordinates from geometry
        #                 coords = word['geometry']
        #                 # Convert coordinates to more readable format
        #                 x1, y1 = coords[0]
        #                 x2, y2 = coords[1]
                        
        #                 words.append({
        #                     'text': word['value'],
        #                     'coords': {
        #                         'y1': float(y1),
        #                         'y2': float(y2),
        #                         'x1': float(x1),
        #                         'x2': float(x2)
        #                     }
        #                 })
        for item in result:
            coords_raw, text, confidence = item
            x1, y1 = coords_raw[0]  # Top-left
            x2, y2 = coords_raw[2]  # Bottom-right
            words.append({
                'text': text,
                'coords': {
                    'y1': float(y1),
                    'y2': float(y2),
                    'x1': float(x1),
                    'x2': float(x2)
                }
            })
        
        # Sort words by y-coordinate first
        words.sort(key=lambda x: x['coords']['y1'])
        
        # Group words that are on the same line
        lines = []
        current_line = [words[0]] if words else []
        
        for word in words[1:]:
            last_word = current_line[-1] if current_line else None
            
            if last_word and self._is_same_line(last_word['coords'], word['coords']):
                current_line.append(word)
            else:
                if current_line:
                    # Sort words in the line by x-coordinate before adding to lines
                    current_line.sort(key=lambda x: x['coords']['x1'])
                    lines.append(current_line)
                current_line = [word]
        
        if current_line:
            # Sort the last line by x-coordinate
            current_line.sort(key=lambda x: x['coords']['x1'])
            lines.append(current_line)
        
        # Convert grouped words to text
        formatted_text = []
        for line in lines:
            line_text = ' '.join(word['text'] for word in line)
            formatted_text.append(line_text)
        
        return formatted_text
    
    def _is_same_line(self, coords1, coords2, tolerance_factor=0.5):
        """
        Check if two words are on the same line based on vertical coordinates
        Using a tolerance factor relative to text height
        """
        height1 = coords1['y2'] - coords1['y1']
        height2 = coords2['y2'] - coords2['y1']
        avg_height = (height1 + height2) / 2
        
        mid1 = (coords1['y1'] + coords1['y2']) / 2
        mid2 = (coords2['y1'] + coords2['y2']) / 2
        
        return abs(mid1 - mid2) < (avg_height * tolerance_factor)

# Example usage
def process_id_card(ocr_result):
    processor = OCRTextProcessor()
    formatted_text = processor.process_ocr_result(ocr_result)
    return formatted_text

In [238]:
class TextEntityExtractor:
    def __init__(self):
        # Define fields with their keywords and tolerance levels
        self.fields = [
            {'name': 'provinsi', 'keywords': ['provinsi'], 'tolerance': 2},
            {'name': 'kabupaten', 'keywords': ['kabupaten', 'kota'], 'tolerance': 2},
            {'name': 'nik', 'keywords': ['nik'], 'tolerance': 1},
            {'name': 'nama', 'keywords': ['nama'], 'tolerance': 1},
            {'name': 'tempat_tgl_lahir', 'keywords': ['tempat/tgl', 'tempat/tgilahir', 'tempat','tompat/tgllah'], 'tolerance': 3},
            # {'name': 'tanggal_lahir', 'keywords': ['tgl', 'tanggal'], 'tolerance': 2},
            {'name': 'jenis_kelamin', 'keywords': ['jenis kelamin', 'kelamin'], 'tolerance': 2},
            {'name': 'alamat', 'keywords': ['alamat'], 'tolerance': 2},
            {'name': 'rt_rw', 'keywords': ['rt/rw', 'rtrw'], 'tolerance': 2},
            {'name': 'kel_desa', 'keywords': ['kel/desa', 'kelurahan', 'desa'], 'tolerance': 2},
            {'name': 'kecamatan', 'keywords': ['kecamatan', 'kec'], 'tolerance': 3},
            {'name': 'agama', 'keywords': ['agama'], 'tolerance': 2},
            {'name': 'status_perkawinan', 'keywords': ['status perkawinan', 'perkawinan'], 'tolerance': 3},
            {'name': 'pekerjaan', 'keywords': ['pekerjaan', 'kerja'], 'tolerance': 3},
            {'name': 'kewarganegaraan', 'keywords': ['kewarganegaraan'], 'tolerance': 4},
            {'name': 'berlaku_hingga', 'keywords': ['berlaku hingga', 'hingga'], 'tolerance': 3}
        ]

    def levenshtein_distance(self, s1, s2):
        """Calculate the Levenshtein distance between two strings"""
        if len(s1) < len(s2):
            return self.levenshtein_distance(s2, s1)

        if len(s2) == 0:
            return len(s1)

        previous_row = range(len(s2) + 1)
        for i, c1 in enumerate(s1):
            current_row = [i + 1]
            for j, c2 in enumerate(s2):
                insertions = previous_row[j + 1] + 1
                deletions = current_row[j] + 1
                substitutions = previous_row[j] + (c1 != c2)
                current_row.append(min(insertions, deletions, substitutions))
            previous_row = current_row

        return previous_row[-1]

    def find_field_match(self, line):
        """Find matching field for a line based on Levenshtein distance"""
        words = line.lower().split()
        if not words:
            return None

        best_match = None
        min_distance = float('inf')
        
        for field in self.fields:
            for keyword in field['keywords']:
                keyword_parts = keyword.lower().split()
                
                # Try matching with first word(s) of line
                for i in range(min(len(words), len(keyword_parts) + 1)):
                    line_part = ' '.join(words[:i+1])
                    distance = self.levenshtein_distance(line_part, keyword)
                    
                    if distance < min_distance and distance <= field['tolerance']:
                        min_distance = distance
                        best_match = field

        return best_match

    def extract_value(self, line, field):
        """Extract value from a line based on field type"""
        # Split line into parts
        parts = line.split()
        
        # Find where the field name ends
        field_end = 0
        for i, part in enumerate(parts):
            for keyword in field['keywords']:
                if self.levenshtein_distance(part.lower(), keyword.lower()) <= field['tolerance']:
                    field_end = i + 1
                    break
        
        # Extract value portion
        value = ' '.join(parts[field_end:]).strip()
        
        # Clean up common artifacts
        value = value.replace(':', '').strip()
        
        return value if value else None

    def extract_entities(self, lines):
        """Extract entities from list of lines"""
        entities = {}
        
        for line in lines:
            # Skip empty lines
            if not line.strip():
                continue
            
            # Find matching field
            field = self.find_field_match(line)
            if field:
                value = self.extract_value(line, field)
                if value:
                    # Special handling for fields that might have multiple parts
                    if field['name'] in entities:
                        if isinstance(entities[field['name']], list):
                            entities[field['name']].append(value)
                        else:
                            entities[field['name']] = [entities[field['name']], value]
                    else:
                        entities[field['name']] = value

        return entities

# Example usage
def extract_id_card_info(lines):
    extractor = TextEntityExtractor()
    return extractor.extract_entities(lines)

In [239]:
# Using the processor
processor = OCRTextProcessor()
formatted_text = processor.process_ocr_result(export)

def preprocess_text(text):
    # Remove non-alphanumeric characters except hyphen (-)
    text = re.sub(r'[^A-Za-z0-9]', ' ', text)
    # Convert to uppercase
    text = text.upper()
    # Remove single characters
    text = ' '.join(word for word in text.split() if len(word) > 1)
    # Remove leading and trailing spaces
    text = text.strip()
    return text

# Apply preprocessing to each string in the list
processed_data = [preprocess_text(item) for item in formatted_text]
processed_data = [item for item in processed_data if item]

processed_data

['PROVINSI SULAWESI SELATAN',
 'KABUPATEN MAROS',
 'NIK 7309012705930004',
 'NAMA MUH DHIAUR RAHMAN',
 'TEMPADIGL LAH MAROS 27 05 1993',
 'JENIS KELAMIN LAKI LAKI GOL DARAH',
 'ALAMAT PP DARUL ISTIQAMAH',
 'RIIRW 001 002',
 'KELLDESA BONTOA',
 'KECAMATAN MANDAI',
 'AGAMA ISUAM',
 'STATUS PERKEW NAN BELUM KAWIN',
 'PEKERJAAN PELAJARIMAHASISWA MAROS',
 'KEWARGANCGARAAN WNI 1710 2017',
 'BERLAKU HINGGE SEUMUR HIDUP']

In [240]:
entities = extract_id_card_info(processed_data)
from datetime import datetime
def post_processing(data):
    import re

    # Helper function to count matching characters
    def count_matching_chars(a, char):
        count = 0
        for c in char:
            if c in a:
                count += 1
        return count
    
    def extract_date_and_place(tempat_tgl_lahir):
        # Regular expression to find 'DD MM YYYY' in the text
        date_match = re.search(r'\b\d{2}\s\d{2}\s\d{4}\b', tempat_tgl_lahir)
        if date_match:
            # Extract the date
            date = date_match.group()
            day, month, year = date.split()
            formatted_date = f"{day}-{month}-{year}"
            # Extract the place (everything before the date)
            place = tempat_tgl_lahir[:date_match.start()].strip()
        else:
            # If no date, use current date and entire input as place
            formatted_date = datetime.now().strftime("%d-%m-%Y")
            place = tempat_tgl_lahir.strip()
        
        return formatted_date, place

    def correct_agama(agama):
        target_words = ["ISLAM", "KRISTEN", "KATOLIK", "HINDU", "BUDDHA", "KONGHUCU"]
        agama = agama.lower()
        match_scores = {word: count_matching_chars(agama, word.lower()) for word in target_words}
        most_likely_agama = max(match_scores, key=match_scores.get)
        return most_likely_agama
    
    def correct_jenis_kelamin(jenis_kelamin):
        target_words = ["LAKI-LAKI", "PEREMPUAN"]
        jenis_kelamin = jenis_kelamin.lower()
        match_scores = {word: count_matching_chars(jenis_kelamin, word.lower()) for word in target_words}
        most_likely_jenis_kelamin = max(match_scores, key=match_scores.get)
        return most_likely_jenis_kelamin
    
    def correct_status_perkawinan(status_perkawinan):
        target_words = ["KAWIN", "BELUM KAWIN", "CERAI HIDUP", "CERAI MATI"]
        status_perkawinan = status_perkawinan.lower()
        match_scores = {word: count_matching_chars(status_perkawinan, word.lower()) for word in target_words}
        most_likely_status_perkawinan = max(match_scores, key=match_scores.get)
        return most_likely_status_perkawinan
    
    def correct_pekerjaan(pekerjaan):
        target_words = [
            "BELUM/TIDAK BEKERJA", "MENGURUS RUMAH TANGGA", "PELAJAR/MAHASISWA", "PENSIUNAN",
            "PEGAWAI NEGERI SIPIL", "TENTARA NASIONAL INDONESIA", "KEPOLISIAN RI", "PERDAGANGAN",
            "PETANI/PEKEBUN", "PETERNAK", "NELAYAN/PERIKANAN", "INDUSTRI", "KONSTRUKSI", "TRANSPORTASI",
            "KARYAWAN SWASTA", "KARYAWAN BUMN", "KARYAWAN BUMD", "KARYAWAN HONORER", "BURUH HARIAN LEPAS",
            "BURUH TANI/PERKEBUNAN", "BURUH NELAYAN/PERIKANAN", "BURUH PETERNAKAN", "PEMBANTU RUMAH TANGGA",
            "TUKANG CUKUR", "TUKANG LISTRIK", "TUKANG BATU", "TUKANG KAYU", "TUKANG SOL SEPATU",
            "TUKANG LAS/PANDAI BESI", "TUKANG JAHIT", "TUKANG GIGI", "PENATA RIAS", "PENATA BUSANA",
            "PENATA RAMBUT", "MEKANIK", "SENIMAN", "TABIB", "PARAJI", "PERANCANG BUSANA", "PENTERJEMAH",
            "IMAM MASJID", "PENDETA", "PASTOR", "WARTAWAN", "USTADZ/MUBALIGH", "JURU MASAK", "PROMOTOR ACARA",
            "ANGGOTA DPR-RI", "ANGGOTA DPD", "ANGGOTA BPK", "PRESIDEN", "WAKIL PRESIDEN",
            "ANGGOTA MAHKAMAH KONSTITUSI", "ANGGOTA KABINET/KEMENTERIAN", "DUTA BESAR", "GUBERNUR",
            "WAKIL GUBERNUR", "BUPATI", "WAKIL BUPATI", "WALIKOTA", "WAKIL WALIKOTA", "ANGGOTA DPRD PROVINSI",
            "ANGGOTA DPRD KABUPATEN/KOTA", "DOSEN", "GURU", "PILOT", "PENGACARA", "NOTARIS", "ARSITEK",
            "AKUNTAN", "KONSULTAN", "DOKTER", "BIDAN", "PERAWAT", "APOTEKER", "PSIKIATER/PSIKOLOG",
            "PENYIAR TELEVISI", "PENYIAR RADIO", "PELAUT", "PENELITI", "SOPIR", "PIALANG", "PARANORMAL",
            "PEDAGANG", "PERANGKAT DESA", "KEPALA DESA", "BIARAWATI", "WIRASWASTA"
        ]
        pekerjaan = pekerjaan.upper()
        match_scores = {word: count_matching_chars(pekerjaan, word) for word in target_words}
        most_likely_pekerjaan = max(match_scores, key=match_scores.get)
        return most_likely_pekerjaan
    
    if 'jenis_kelamin' in data:
        data['jenis_kelamin'] = correct_jenis_kelamin(data['jenis_kelamin'])
    if 'agama' in data:
        data['agama'] = correct_agama(data['agama'])
    # if 'pekerjaan' in data:
    #     data['pekerjaan'] = correct_pekerjaan(data['pekerjaan'])
    if 'status_perkawinan' in data:
        data['status_perkawinan'] = correct_status_perkawinan(data['status_perkawinan'])
    if 'tempat_tgl_lahir' in data:
        extracted_date, extracted_place = extract_date_and_place(data['tempat_tgl_lahir'])
        data['tanggal_lahir'] = extracted_date
        data['tempat_lahir'] = extracted_place
    return data

result = post_processing(entities)
result

{'provinsi': 'SULAWESI SELATAN',
 'kabupaten': 'MAROS',
 'nik': '7309012705930004',
 'nama': 'MUH DHIAUR RAHMAN',
 'tempat_tgl_lahir': 'LAH MAROS 27 05 1993',
 'jenis_kelamin': 'LAKI-LAKI',
 'alamat': 'PP DARUL ISTIQAMAH',
 'rt_rw': '001 002',
 'kel_desa': 'BONTOA',
 'kecamatan': 'MANDAI',
 'agama': 'ISLAM',
 'status_perkawinan': 'BELUM KAWIN',
 'pekerjaan': 'PELAJARIMAHASISWA MAROS',
 'kewarganegaraan': 'WNI 1710 2017',
 'berlaku_hingga': 'SEUMUR HIDUP',
 'tanggal_lahir': '27-05-1993',
 'tempat_lahir': 'LAH MAROS'}

In [241]:
a = "widi"
char = "wd"

# count how many same characters founded are in the string
count = 0
for i in range(len(char)):
    if char[i] in a:
        count += 1
count

2

In [242]:
import re

# Sample data
data = {
    'provinsi': 'SULAWESI SELATAN',
    'kabupaten': 'MAROS',
    'nik': '7309012705930004',
    'nama': 'MUH DHIAUR RAHMAN',
    'tempat_tgl_lahir': 'LAH MAROS 27 05 1993',
    'jenis_kelamin': 'LAKI-LAKI',
    'alamat': 'PP DARUL ISTIQAMAH',
    'rt_rw': '001 002',
    'kel_desa': 'BONTOA',
    'kecamatan': 'MANDAI',
    'agama': 'ISLAM',
    'status_perkawinan': 'BELUM KAWIN',
    'pekerjaan': 'TENTARA NASIONAL INDONESIA',
    'kewarganegaraan': 'WNI 1710 2017',
    'berlaku_hingga': 'SEUMUR HIDUP'
}

# Extract 'tempat_tgl_lahir'
tempat_tgl_lahir = data['tempat_tgl_lahir']

# Regular expression to extract the date in 'DD MM YYYY' format
date_match = re.search(r'\b\d{2}\s\d{2}\s\d{4}\b', tempat_tgl_lahir)

# Extract and display the date
if date_match:
    date = date_match.group()
    print("Extracted date:", date)
else:
    print("Date not found in 'tempat_tgl_lahir'")


Extracted date: 27 05 1993


In [243]:
# here the result:
# provinsi: JAWA TENGAH kabupaten: . PURBALINGGA nik: - 330315040458000 nama: IFTAH SADJAD AHMADI tempat_lahir: PURBAUNGGA tanggal_lahir: 04-06-1998 jenis_kelamin: LAKRAKE Gol Darah a alamat: PERLMEDMIPAOAMMAGNFS rt_rw: 001003 kel_desa: BOJANEGARA kecamatan: PADAMARA agama: ISLAM status_perkawinan: BELUMKAWIN PUHBALINGGA pekerjaan: PELLARMAHASISHA 19092015 kewarganegaraan: WNI berlaku_hingga: SEUMUR RIDUP

# do validation like:
# 1. there is no number in "pekerjaan", so if there is detected number, remove the number
# 2. in status perkawinan mostly the value is : belum kawin, kawin, cerai mati, cerai hidup. check if tolerance in that value. in my case it detected BELUMKAWIN PUHBALINGGA. remove else the value i gave you earlier (PUHBALINGGA)
# 3. Gol darah is completly new key